# enriched (main) Persons with Job
* write Persons (main and male) to .txt file
* use this file to annotate Jobs within prodigy
* train a job-model
* run it and annotate Persons

In [ ]:
import ast
import random
from spacytei.train import batch_train

In [ ]:
male_main_person = Person.objects.filter(gender='männlich').filter(is_main_person__isnull=False)
male_main_person.count()

In [ ]:
items = [x.written_name for x in male_main_person]

In [ ]:
items = sorted(iter(items), key=lambda k: random.random())

In [ ]:
filename = "person__male_main.txt"

In [ ]:
with open(filename, 'w', encoding="utf-8") as f:
    for x in items:
        f.write("{}".format(x) + '\n')

In [ ]:
file = r"C:\Users\pandorfer\Documents\Redmine\prodigy\invs\invs_jobs.jsonl"

In [ ]:
with open(file) as f:
    TRAIN_DATA = f.readlines()

In [ ]:
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
len(train_data)
train_data[4]

In [ ]:
batch_train(train_data=train_data, output_dir='./data/jobs_model', new_label='JOB', eval_split=0.2, n_iter=8)

In [ ]:
import spacy

In [ ]:
nlp = spacy.load(r"./data/jobs_model")

In [ ]:
job_scheme, _ = SkosCollection.objects.get_or_create(name="Berufe")

In [ ]:
for x in male_main_person:
    doc = nlp("{}".format(x.written_name))
    for ent in doc.ents:
        if ent.label_ == "JOB":
            job, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(ent.text)
            )
            job.collection.add(job_scheme)
#             print(ent.text)
            x.profession.add(job)
            break
    x.save()

### remarks:
the code/model above produces a lot of false positivs (all in all 420 Concepts). Ideally one could define a threashold of some NER confidence score but such thing does not exist [see this issue](https://github.com/explosion/spaCy/issues/881). Potential work arounds could be
1. only run on male main persons
2. manually go to the list of all 'professions' and delete wrong ones
3. use the 'professions' manually annotated and annotate Jobs by simple string match
4. combination of the two options

#### tryouts
* delete all Jobs and only annotate male main persons

In [ ]:
SkosConcept.objects.filter(collection__name="Berufe").delete()

In [ ]:
for x in Person.objects.filter(gender='männlich').filter(is_main_person__isnull=False):
    doc = nlp("{}".format(x.written_name))
    for ent in doc.ents:
        if ent.label_ == "JOB":
            job, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(ent.text)
            )
            job.collection.add(job_scheme)
#             print(ent.text)
            x.profession.add(job)
            break
    x.save()

#### annotating only male main persons
* yields much better results
* therefore it make sense to train an additional model for non-main and non-adm persons

In [ ]:
related_person = Person.objects.filter(gender='männlich').filter(is_related_person__isnull=False)
related_person.count()

In [ ]:
items = [x.written_name for x in related_person]
items = sorted(iter(items), key=lambda k: random.random())
filename = "person__male_related.txt"
with open(filename, 'w', encoding="utf-8") as f:
    for x in items:
        f.write("{}".format(x) + '\n')

In [ ]:
file = r"C:\Users\pandorfer\Documents\Redmine\prodigy\invs\invs_jobs_related.jsonl"
with open(file) as f:
    TRAIN_DATA = f.readlines()
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
len(train_data)
train_data[4]

In [ ]:
batch_train(train_data=train_data, output_dir='./data/jobs_related_model', new_label='JOB', eval_split=0.2, n_iter=8)

In [ ]:
nlp = spacy.load(r"./data/jobs_related_model")

In [ ]:
for x in related_person:
    doc = nlp("{}".format(x.written_name))
    for ent in doc.ents:
        if ent.label_ == "JOB":
            job, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(ent.text)
            )
            job.collection.add(job_scheme)
#             print(ent.text)
            x.profession.add(job)
            break
    x.save()